<a href="https://colab.research.google.com/github/LjBLincoln/Machine_Learning/blob/master/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as  tf

from tensorflow.examples.tutorials.mnist import input_data

#get MNIST data set
mnist = input_data.read_data_sets('./tmp/data/mnist', one_hot=True)
#input module
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

#softmax layer
with tf.name_scope('softmax_layer'):
    with tf.name_scope('weights'):
        weights = tf.Variable( tf.zeros( [ 784, 10 ] ) )
        tf.summary.histogram('weights' ,weights)
    with tf.name_scope('biases'):
        biases =  tf.Variable( tf.zeros( [ 10 ] ) )
        #tf.summary.histogram('biases' ,biases)
    with tf.name_scope('Wx_plus_b'):
        y = tf.matmul( x,  weights ) + biases

#cross entropy
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits( labels=y_, logits= y)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
        tf.summary.scalar('cross_entropy', cross_entropy)

#Optimizer
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
#accuracy
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

# crate interactive session
sess = tf.InteractiveSession()

#create FileWriter and loadd graph
train_writer = tf.summary.FileWriter('./tmp/summary/mnist'+'/train', sess.graph)
test_writer = tf.summary.FileWriter('./tmp/summary/mnist'+'/test')
#init global variables
tf.global_variables_initializer().run()

def feed_dict(train):
    if train:
        xs, ys = mnist.train.next_batch(100, fake_data=False)
    else:
        xs, ys = mnist.test.images, mnist.test.labels
    return {x:xs, y_:ys}

for i in range(1000):
    if i %  10 == 0:
        summary, acc = sess.run([merged,accuracy], feed_dict=feed_dict(True))
        test_writer.add_summary(summary,i)
        print('Accuracy at step %s : %s ' % (i,acc ))
    else :
        if i % 100 == 99:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged,train_step],
                                  feed_dict=feed_dict(False),
                                  options=run_options,
                                  run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            print('Adding run metadata fro ', i)
        else:
            summary, _ = sess.run([merged, train_step] , feed_dict= feed_dict(True))
            train_writer.add_summary(summary, i)
#close FileWriter
train_writer.close()

test_writer.close()

#https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format("./tmp/summary/mnist"))

get_ipython().system_raw('./ngrok http 6006 &')


! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./tmp/data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./tmp/data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./tmp/data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./tmp/data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as offi